<a href="https://colab.research.google.com/github/SanskarGithub07/FDS-Implementation/blob/main/fds_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Collection and Cleaning
- Collecting data from kaggle and using the PennTreebank dataset for our implementation of the model.
- Cleaning the data for better representation and model accuracy

In [1]:
import pandas as pd
import numpy as np
import collections

In [2]:
def read_words(filename):
    with open(filename, "r", encoding="utf-8", errors="replace") as f:
        return f.read().replace("\n", "<eos>").split()

In [3]:
def build_vocab(filename):
    data = read_words(filename)
    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = zip(*count_pairs)
    word_to_id = {word: i for i, word in enumerate(words)}
    return word_to_id

In [4]:
def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [5]:
train_path = "ptb.train.txt"
valid_path = "ptb.valid.txt"
test_path = "ptb.test.txt"
train_path, valid_path, test_path

('ptb.train.txt', 'ptb.valid.txt', 'ptb.test.txt')

In [6]:
def load_ptb_dataset():

    word_to_id = build_vocab(train_path)

    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocab_size = len(word_to_id)

    return train_data, valid_data, test_data, vocab_size, word_to_id

train_data, valid_data, test_data, vocab_size, word_to_id = load_ptb_dataset()

In [7]:
print(f"Vocabulary size: {vocab_size}",f"Train data size: {len(train_data)}",
f"Valid data size: {len(valid_data)}", f"Test data size: {len(test_data)}")
print(f"\n\nSample word to id mapping: {list(word_to_id.items())[:5]}")

Vocabulary size: 10000 Train data size: 929589 Valid data size: 73760 Test data size: 82430


Sample word to id mapping: [('the', 0), ('<unk>', 1), ('<eos>', 2), ('N', 3), ('of', 4)]


In [8]:
def id_to_word(id_list, word_to_id=word_to_id):
    id_to_word_dict = {v: k for k, v in word_to_id.items()}
    return [id_to_word_dict[id_] for id_ in id_list]

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


In [9]:
print(train_data[0:100])

[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996, 9997, 9998, 9999, 2, 9256, 1, 3, 72, 393, 33, 2133, 0, 146, 19, 6, 9207, 276, 407, 3, 2, 23, 1, 13, 141, 4, 1, 5465, 0, 3081, 1596, 96, 2, 7682, 1, 3, 72, 393, 8, 337, 141, 4, 2477, 657, 2170, 955, 24, 521, 6, 9207, 276, 4, 39, 303, 438, 3684, 2, 6, 942, 4, 3150, 496, 263, 5, 138, 6092, 4241, 6036, 30, 988, 6, 241, 760, 4, 1015, 2786, 211, 6, 96, 4]


In [13]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from collections import defaultdict

class SVD2Tagger:
    def __init__(self, w1=1000, r1=100, k1=500, r2=300, k2=45):
        """
        Initialize SVD2 tagger with parameters
        w1: number of most frequent words for context
        r1: rank for first SVD
        k1: number of clusters for first clustering
        r2: rank for second SVD
        k2: final number of POS tags
        """
        self.w1 = w1
        self.r1 = r1
        self.k1 = k1
        self.r2 = r2
        self.k2 = k2

    def build_context_matrices(self, tokens, vocab_size, word_to_id):
        """Build left and right context matrices"""
        # Get most frequent word indices
        word_counts = np.bincount(tokens)
        top_words = np.argsort(word_counts)[-self.w1:]

        # Initialize context matrices
        L = np.zeros((vocab_size, self.w1))
        R = np.zeros((vocab_size, self.w1))

        # Fill context matrices
        for i in range(len(tokens)-1):
            curr_word = tokens[i]
            # Right context
            next_word = tokens[i+1]
            if next_word in top_words:
                R[curr_word, np.where(top_words == next_word)[0][0]] += 1

            # Left context
            if i > 0:
                prev_word = tokens[i-1]
                if prev_word in top_words:
                    L[curr_word, np.where(top_words == prev_word)[0][0]] += 1

        return L, R

    def svd_transform(self, matrix, rank):
        """Perform reduced rank SVD and return transformed matrix"""
        U, S, Vt = np.linalg.svd(matrix, full_matrices=False)
        # Keep only top rank singular values
        S = np.diag(S[:rank])
        U = U[:, :rank]
        return normalize(U @ S)

    def cluster_descriptors(self, descriptors, n_clusters, word_counts):
        """Perform weighted k-means clustering"""
        # Initialize centroids with most frequent words
        top_indices = np.argsort(word_counts)[-n_clusters:]
        init_centroids = descriptors[top_indices]

        kmeans = KMeans(n_clusters=n_clusters, init=init_centroids, n_init=1)
        return kmeans.fit_predict(descriptors)

    def fit(self, tokens, vocab_size, word_to_id):
        """Perform full SVD2 training process"""
        # First pass
        print("Building initial context matrices...")
        L1, R1 = self.build_context_matrices(tokens, vocab_size, word_to_id)

        print("Performing first SVD transformation...")
        L1_transformed = self.svd_transform(L1, self.r1)
        R1_transformed = self.svd_transform(R1, self.r1)

        # Concatenate descriptors
        descriptors1 = np.hstack([L1_transformed, R1_transformed])

        print("Performing first clustering...")
        word_counts = np.bincount(tokens)
        first_clusters = self.cluster_descriptors(descriptors1, self.k1, word_counts)

        # Second pass
        print("Building refined context matrices...")
        L2 = np.zeros((vocab_size, self.k1))
        R2 = np.zeros((vocab_size, self.k1))

        # Fill second pass context matrices using cluster assignments
        for i in range(len(tokens)-1):
            curr_word = tokens[i]
            # Right context
            next_word = tokens[i+1]
            R2[curr_word, first_clusters[next_word]] += 1

            # Left context
            if i > 0:
                prev_word = tokens[i-1]
                L2[curr_word, first_clusters[prev_word]] += 1

        print("Performing second SVD transformation...")
        L2_transformed = self.svd_transform(L2, self.r2)
        R2_transformed = self.svd_transform(R2, self.r2)

        # Final clustering
        print("Performing final clustering...")
        descriptors2 = np.hstack([L2_transformed, R2_transformed])
        self.final_clusters = self.cluster_descriptors(descriptors2, self.k2, word_counts)

        return self.final_clusters

    def get_cluster_examples(self, tokens, word_to_id, n_examples=5):
        """Get example words for each cluster"""
        id_to_word = {v: k for k, v in word_to_id.items()}
        cluster_examples = defaultdict(list)

        for word_id in range(len(self.final_clusters)):
            cluster = self.final_clusters[word_id]
            if word_id in id_to_word:
                word = id_to_word[word_id]
                cluster_examples[cluster].append(word)

        # Print top n examples for each cluster
        for cluster in sorted(cluster_examples.keys()):
            examples = cluster_examples[cluster][:n_examples]
            print(f"Cluster {cluster}: {', '.join(examples)}")

# Example usage
tagger = SVD2Tagger(w1=1000, r1=100, k1=500, r2=300, k2=45)
clusters = tagger.fit(train_data, vocab_size, word_to_id)
print("\nCluster Examples:")
tagger.get_cluster_examples(train_data, word_to_id)

Building initial context matrices...
Performing first SVD transformation...
Performing first clustering...
Building refined context matrices...
Performing second SVD transformation...
Performing final clustering...

Cluster Examples:
Cluster 0: trading, business, companies, quarter, prices
Cluster 1: earned, totaled, totaling, approximately, fined
Cluster 2: compared, along, venture, familiar, met
Cluster 3: recent, early, cash, congress, late
Cluster 4: year, share, major, very, few
Cluster 5: international, boston, pacific, communications, brothers
Cluster 6: co., securities, trade, insurance, management
Cluster 7: company, market, group, government, time
Cluster 8: or, shares, common, yen, canadian
Cluster 9: new, u.s., stock, first, two
Cluster 10: expected, back, according, plans, going
Cluster 11: n't, will, would, also, could
Cluster 12: not, no, just, being, less
Cluster 13: investment, own, analyst, annual, life
Cluster 14: fulton, u.s.a, wedd, aer, banknote
Cluster 15: mr., h